In [ ]:
#download the retinanet api(folders)

In [ ]:
!git clone 'https://github.com/fizyr/keras-retinanet.git'

In [ ]:
##going to retinanet folder

In [ ]:
cd /content/drive/MyDrive/keras-retinanet 

/content/drive/MyDrive/keras-retinanet


In [ ]:
##installing the requirements

In [ ]:
!pip install . --user

Processing /content/drive/MyDrive/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-1.0.0-cp37-cp37m-linux_x86_64.whl size=168427 sha256=5dc110018ff7a4b29c35810de8aed9ee096fb8b08de1dd3c567c6c8cf285b930
  Stored in directory: /root/.cache/pip/wheels/16/cc/dc/f6b0df540a02e47f927dd0d9ec6e3f98a0e724f68798846d24
Successfully built keras-retinanet
  Found existing installation: keras-retinanet 1.0.0
    Uninstalling keras-retinanet-1.0.0:
      Successfully uninstalled keras-retinanet-1.0.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!python setup.py build_ext --inplace

running build_ext
skipping 'keras_retinanet/utils/compute_overlap.c' Cython extension (up-to-date)
copying build/lib.linux-x86_64-3.7/keras_retinanet/utils/compute_overlap.cpython-37m-x86_64-linux-gnu.so -> keras_retinanet/utils


In [ ]:
# Download the helper library from https://www.twilio.com/docs/python/install
#for sending messages

In [ ]:
!pip install twilio

In [ ]:
'''For every new object detected in the current frame:
    1.1 For every old object from the previous frame:
        1.1.1 Find an old object such that the distance between this old object and new object is the least
              among all combinations of old object and new object pairs.
    1.2 The old object with the least distance to the new object is most certainly the same object. This 
        is because an object can only move so far between subsequent frames so this distance will almost
        always be smaller than the distance between 2 different objects.
    1.3 Assign the new object the index of the corresponding old object.
    1.4 Update new object's number of frames detected and vectors accordingly
    
    we check if an old object has already been assigned to a new object. If this is the case and there
is a conflict between assigning two new objects to a single old object, we will compare the distances between
the two pairs and decide which new object the old object actually corresponds to. The incorrect new object is 
then marked as "not found" and will later be assigned a completely new index.'''


#importing packages
import tensorflow as tf
from tensorflow import keras
import cv2                                    #for image processing
import numpy as np
from google.colab.patches import cv2_imshow    #to show images in colab
from tensorflow.keras.models import load_model 

#packages from retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

import os
from os.path import isfile, join

from twilio.rest import Client

model_path="inference_graphs/crash_detection_model.h5"    #weights location
model=models.load_model(model_path, backbone_name='resnet50') #load the model

pat='/content/drive/MyDrive/keras-retinanet/videos/junction_crashes1.mp4'   #video source

source=cv2.VideoCapture(pat)          #start video processing


##setting all parameters for video 
# l=list(pat.split('/'))[-1]
# l=list(l.split('.'))[0]
# out_path='/content/drive/MyDrive/keras-retinanet/out_videos/'+l+".mp4"  #video output location
# fps = 16  #video frames per second
# #getting size of 1st frame to set size for video
# ret,img=source.read()   
# rows = img.shape[0]
# cols = img.shape[1]
# if rows < cols:
#     padding = int((cols - rows) / 2)
#     img = cv2.copyMakeBorder(img, padding, padding, 0, 0, cv2.BORDER_CONSTANT, (0, 0, 0))
# elif rows > cols:
#     padding = int((rows - cols) / 2)
#     img = cv2.copyMakeBorder(img, 0, 0, padding, padding, cv2.BORDER_CONSTANT, (0, 0, 0))
# draw = img.copy()
# height, width, layers = draw.shape #getting dimension of image
# size = (width,height)
# out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc(*'DIVX'), fps, size) #start video input
# source=cv2.VideoCapture(pat)          #restart video processing again

'''frame_objects structure for EACH object:
Index 0: Tuple for the midpoint of the object
Index 1: Number of frames object has been consecutively detected for
Index 2: queue to store midpoints of the object over the past 5 frames. Used for vector calculation
Index 3: for previous frame this stores the array index of the new object that this old object has been assigned to.
         for current frame this stores the index of previous frame object to which this new object is attached
         This is used to fix any conflicts between 2 or more new objects that have been assigned to the same
         old object(this happens when a new object comes). 
Index 4: Magnitude of vector for the object of current frame'''
images=[]                             #array to store each frame to build video
prev=[]                               #stores objects of previous frame
cur=[]                                #stores objects of  current frame


#function to calculate distance between two points
def dist(p1,p2):
  x=(p1[0]-p2[0])**2
  y=(p1[1]-p2[1])**2
  return (x+y)**0.5


f=5
#message sending funtion
#dont use unnecessaryly
def notify(id):
  global f
  if f>0:
    print('inside')
    f-=1
    # Your Account Sid and Auth Token from twilio.com/console
    client = Client("AC597573dbb155e6900c95ba0a07315393", "f14e13ad06f422680560d5470798c6d6")  #initialize client with(Account Sid,Auth Token)
    msg="--\nCRASH OCCURED  \n view crash image having ID **img{}_jpg** at : \n https://drive.google.com/drive/folders/1PpyH8W6tvbGurWS5hrgdWnvxGU3jQFrp?usp=sharing".format(id)
    client.messages.create(from_="+17028304663",
                                      to="+919247446977",
                                      body=msg ,
                                     
                                      )

count=0     #frame count
count_crash=0   #crash frame count

#loop to process video frame by frame
while(True):
    
    ret,img=source.read()   #reading each frame

    if(ret==False):         #if frame not read =>end of video so break loop
      break
   
    # Making input image square by padding it. This is because our model was trained on square
    # images. Changing the aspect ratio of input images would greatly reduce accuracy
    rows = img.shape[0]
    cols = img.shape[1]
    if rows < cols:
        padding = int((cols - rows) / 2)
        img = cv2.copyMakeBorder(img, padding, padding, 0, 0, cv2.BORDER_CONSTANT, (0, 0, 0))
    elif rows > cols:
        padding = int((rows - cols) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, padding, padding, cv2.BORDER_CONSTANT, (0, 0, 0))
    
    
    draw = img.copy() #storing a copy of frame to use for output

    #preprocessing image to apply retinanet
    img = preprocess_image(img)
    img, scale = resize_image(img,min_side=600) #minsize by default 800 but we  have imgs of 600,600 shape 
    
    #apply retinanet to each frame
    '''Where boxes are shaped  (left,top,bottom,right)),
     scores is shaped (None, None) (classification score for each label) 
     and labels is shaped (None, None) (label corresponding to the score)'''

    boxes, scores, labels = model.predict_on_batch(np.expand_dims(img, axis=0)) #prediction using retinanet
    boxes /= scale   # Adjusting scale of bounding boxes since our frame is resized to 600p
    
    is_crash=False  #flag used to indicate if any one vehicle is involved in crash for in THIS frame 
    

    #traversing every object in the frame
    for box, score in zip(boxes[0], scores[0]):
        # We break if the score of any of our detections is below 0.95. This is because
        # all detections are sorted in descending order in terms of their scores.
        if score < 0.95:
            break
        #getting midpoint of the bounding box
        mid_x = int((box[2] + box[0]) / 2)
        mid_y = int((box[3] + box[1]) / 2)

        #creating the frame_object structure mentioned above 
        # for each object and add them to cur array 
        cur.append([(mid_x,mid_y),1,[(mid_x,mid_y)],-1,0])
    
    ##check if this is initial frame(i.e if already some objects exists in prev frame or not)
    if len(prev)!=0:
      #iterate through CURRENT frame objects
      for i,obj in enumerate(cur):
        m=float('inf')
        pre_ind=-1
        #for each cur object check distance with every prev obj
        for e,pobj in enumerate(prev):
          if dist(pobj[0],obj[0])<m:
            m=dist(pobj[0],obj[0])
            pre_ind=e
        
        # pre_ind contains the INDEX of object in prev frame which has min_dict to cur object in cur frame
        
        #if that previous object is not assigned to any other cur object,then establish the dual(both sides) link
        if (prev[pre_ind][3]==-1): #checking if the index 3: is -1 or already assigned
          prev[pre_ind][3]=i  #establish connection from prev to curr
          cur[i][3]=pre_ind   #establish connection from curr to prev 

        #if that previous object is already assigned to any new object
        #check the distance between 1:previous object and its already assigned old cur_frame object
        #                      and  2:previous object and the new cur_frame object 
        #establish link between objects having minimum distance                              
        else:
          #already assigned object has min_dist do nothing 
          #else change the links between previous object and cur objects
          if (dist(prev[pre_ind][0],cur[i][0])<dist(prev[pre_ind][0],cur[prev[pre_ind][3]][0])):  #check if distance between previous object and the new cur object is less than previous object and its already assigned old cur object 

            cur[i][3]=pre_ind  #establish link from new cur_frame object to min_dist prev_frame object obtained above(pre_ind)
            cur[prev[pre_ind][3]][3]=-1  #remove link from old cur_frame object and prev_frame object obtained above(pre_ind)
            prev[pre_ind][3]=i  #establish link from prev object to new cur_frame object

        
        #fill the remaining feilds in cur_frame objects only if they are linked to previous frame
        #else they are considered as new objects(and nothing more need to be done)
      for i,obj in enumerate(cur):
        if cur[i][3]!=-1: #for cur_frame objects which has a link to previous frame 

            prev_ind=cur[i][3]  ##geting prev_obj index to which cur obj is linked
            
            cur[i][1]=prev[prev_ind][1]+1  #incrementing frame count of cur object= prev+1
            

            cur[i][2]=prev[prev_ind][2].copy() #copying the mid_points list of prev_frame linked object 
            
            cur[i][2].append(cur[i][0])  #updating the mid points list by adding curent mid point

            
            #if frame count =5 initialize the magnitude 
            '''The vector for every object is calculated from the object's midpoint at the 1st frame, and the midpoint at
                the 5th frame. We store these midpoints in a deque inside prev_frame_objects and/or cur_frame_objects. 
                The deque is kept updated every frame by removing the oldest midpoint and adding the latest midpoint'''
            if cur[i][1]==5:     #if obj is present for k consecutive frames then draw vector 
                temp=cur[i][2].copy() #copying mid_points array in temp variable
                vector=[temp[-1][0]-temp[0][0],temp[-1][1]-temp[0][1]] #constructing vector between 1st point in mid_point array and last(latest midpoint) point in array
                cur[i][4]=(vector[0]**2+vector[1]**2)**(1/2) #getting the magnitude of vector(distance between points) and store in index 4 of object

                next_p=(2*vector[0]+cur[i][0][0],2*vector[1]+cur[i][0][1])  ##calculation of next projected point 
                cv2.circle(draw, cur[i][0], 5, (0, 0, 255), -1)     ##draw circle around vehicle
                cv2.line(draw, cur[i][0], next_p, (0, 255, 0), 2)   ##draw vector (from cur mid point to projected point)
            
            elif cur[i][1]>5:
                cur[i][2].pop(0) #remove the initial mid point as we need only the recent 3 mid points
                #same as above
                temp=cur[i][2]
                vector=[temp[-1][0]-temp[0][0],temp[-1][1]-temp[0][1]]
                cur[i][4]=(vector[0]**2+vector[1]**2)**(1/2)

                next_p=(2*vector[0]+cur[i][0][0],2*vector[1]+cur[i][0][1])  ##calculation of next projected point 
                cv2.circle(draw, cur[i][0], 5, (0, 0, 255), -1)     ##draw circle around vehicle
                cv2.line(draw, cur[i][0], next_p, (0, 255, 0), 2)   ##draw vector

                #check the difference in prev vector magnitude and curr vector magnitude 
                #if greater than threshold report as accident  
                if abs(cur[i][4]-prev[prev_ind][4])>11:     ##change in magnitude for crash to occur
                    cv2.circle(draw, cur[i][0], 40, (0, 0, 255), 4)   ##draw circle over crash
                    is_crash=True  #set crash flag as true
                    
    # print(prev)
    # print(cur)
    #by checking for all objects above if any crash occurs display text and store frame              
    if is_crash: #check if crash occured
      cv2.putText(draw,"CRASH DETECTED", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2) ##print crash_detected on img
      
      if os.path.exists("/content/drive/MyDrive/keras-retinanet/crash_pics/img%d.jpg" % count_crash): #check if old crash image with same id exists already
        os.remove("/content/drive/MyDrive/keras-retinanet/crash_pics/img%d.jpg" % count_crash) #remove the old crash pic

      cv2.imwrite(r"/content/drive/MyDrive/keras-retinanet/crash_pics/img%d.jpg" % count_crash, draw) #storing crash images
      #notify(count_crash)  #calling the notification function
      count_crash+=1  #increment crash_count

    #remove all links from cur to prev ..so when cur becomes prev.. all links are unset for future use
    for ind in range(len(cur)):
      cur[ind][3]=-1
    
    prev=cur.copy() #copy cur to prev
    cur=[]   #set cur to empty
    
    #cv2_imshow(draw) #display image
    
    #print(count)

    #building video
      
    cv2.imwrite(r"/content/drive/MyDrive/keras-retinanet/temp/img%d.jpg" % count, draw) #storing frame for future use in video
    images.append("/content/drive/MyDrive/keras-retinanet/temp/img{}.jpg".format(count)) #storing frame address in array 
    # cv2.imwrite(r"/content/drive/MyDrive/keras-retinanet/temp/img.jpg", draw)
    # # temp_frame = cv2.imread(r"/content/drive/MyDrive/keras-retinanet/temp/img.jpg")  #reading the current frame  again 
    # # out.write(temp_frame)  #writing the frame to video
    # # os.remove(r"/content/drive/MyDrive/keras-retinanet/temp/img.jpg") #remove the stored frame
    count+=1  #increment frame count  to store image in temp
    
    key=cv2.waitKey(1)
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()  #close video


#buliding video

l=list(pat.split('/'))[-1]
l=list(l.split('.'))[0]
out_path='/content/drive/MyDrive/keras-retinanet/out_videos/'+l+".mp4"  #video output location
#print(out_path)
fps = 16  #video frames per second

#print(images)  #print frame_address list
img = cv2.imread(images[0])  #read image
height, width, layers = img.shape #getting dimension of image
size = (width,height)
out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc(*'DIVX'), fps, size) #start video input
for i in range(len(images)):  #traverse every address and add image
    #print(i,images[i])
    try:
      img = cv2.imread(images[i])
      out.write(img)  #writing to video
      os.remove(images[i])  #remove image from temp folder
    except:
      continue

 
out.release()  #close video


In [ ]:
pwd

'/content/drive/My Drive/keras-retinanet'

In [ ]:
##code to empty the crash_pics directory
import os
dirpath="/content/drive/MyDrive/keras-retinanet/crash_pics/"
l=os.walk(dirpath)
for root, dirs, files in l:
    for file in files:
        os.remove(os.path.join(root, file))
